Running requires having the data on Google Drive or uploading the data to Colab via the left menu and setting the data path correctly.

To use TPU, set "tpu" to True and hardware accelerator to "TPU" from Edit -> Notebook Settings

In [1]:
seed = 42
import pandas as pd
import numpy as np
import random
np.random.seed(seed)
random.seed(seed)
tpu = False
from google.colab import drive
drive.mount('/gdrive')
data_path = '/gdrive/My Drive/Colab Notebooks/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
import os
import tensorflow as tf
# This address identifies the TPU we'll use when configuring TensorFlow.
if tpu:
  TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  tf.logging.set_verbosity(tf.logging.INFO)

def tpu_compatibilitate(model):
  if tpu: return tf.contrib.tpu.keras_to_tpu_model(
      model, strategy=tf.contrib.tpu.TPUDistributionStrategy(
              tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))
  else: return model

Read data

In [3]:
data = pd.read_csv(data_path + 'EEG_data.csv')
data.columns

Index(['SubjectID', 'VideoID', 'Attention', 'Mediation', 'Raw', 'Delta',
       'Theta', 'Alpha1', 'Alpha2', 'Beta1', 'Beta2', 'Gamma1', 'Gamma2',
       'predefinedlabel', 'user-definedlabeln'],
      dtype='object')

The labels for confusion are the same for each video by subject

In [0]:
for subjId in set(data.SubjectID):
  for vidId in set(data.VideoID):
    assert data.query('SubjectID == {} and VideoID == {}'
                     .format(subjId, vidId))['user-definedlabeln'].mean() in (0.0, 1.0)

Load subtitle vectors  


In [0]:

import numpy as np
"""
from csv
"""
#vid_dfs = pd.concat([pd.read_csv(notebook_path + 'subtitles/vid_{}_elmo_embedded_subs.csv'.format(i))
#                     for i in range(10)], ignore_index=True
#                   ).sort_values(['SubjectID', 'VideoID']).reset_index(drop=True)

#vec_cols = [str(x) for x in range(1024)]

#sub_vecs = vid_dfs[vec_cols].values.astype('float32')

"""
save/load from npy
"""
sub_vec_path = data_path + 'subtitle_vecs.npy'
#np.save(sub_vec_path, sub_vecs)
sub_vecs = np.load(sub_vec_path)
sub_vec_dim = sub_vecs.shape[1]

"""
Make a dataset of original data combined with sub vecs 
"""
dataset = np.hstack((data.values.astype('float32'), sub_vecs))

PCA to reduce subtitle vector dimensions. Speeds up training and also has the potential to increase performance

In [0]:
from sklearn.decomposition import PCA
"""
PCA to reduce dimension of the word average vectors (might give better results)
"""
sub_vec_dim = 12
pca = PCA(n_components=sub_vec_dim)
pcad_sub_vecs = pca.fit_transform(sub_vecs)
dataset = np.hstack((data.values.astype('float32'), pcad_sub_vecs))

In [7]:
dataset.shape

(12811, 27)

Preprocessing as is done in https://github.com/mehmani/DNNs-for-EEG-Signals/blob/master/DNNforEEFSignals.ipynb

In [8]:

import numpy as np
from sklearn.preprocessing import MinMaxScaler

def NormSignal(S, I):
    #normalize features
    S=S.reshape(-1, 1)
    if I not in [0, 1, 13, 14]:
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled = scaler.fit_transform(S)
        scaled = scaled
    else:
        scaled = S
    return scaled.reshape(-1).tolist()

NormDataG = np.array([NormSignal(dataset[:,i], i) for i in range(dataset.shape[1])]).T
print(NormDataG.shape)

(12811, 27)


Additional metrics besides accuracy to have more information on model performance 

In [0]:
from sklearn.metrics import roc_auc_score, f1_score
from tensorflow.keras.callbacks import Callback

class f1_auc_callback(Callback):
    def __init__(self, X_test, y_test, f1s, roc_aucs):
        self.X_test = X_test
        self.y_test = y_test.flatten()
        self.f1s = f1s
        self.roc_aucs = roc_aucs

    def on_train_end(self, epoch, logs={}):
        y_pred = self.model.predict_proba(self.X_test, verbose=0).flatten()
        roc_test = roc_auc_score(self.y_test, y_pred)
        f1_test = f1_score(self.y_test, np.round(y_pred))
        #print('\r rocauc %s f1 %s' % (str(roc_test), str(f1_test)), end=10*' ' + '\n')
        self.f1s.append(roc_test)
        self.roc_aucs.append(f1_test)
        return 


Model from the paper "confused or not confused"

In [0]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.python.keras.layers import BatchNormalization
from tensorflow.python.keras.layers import Input, LSTM, Bidirectional, Dense, Flatten, Dropout, TimeDistributed, Conv2D, MaxPooling2D, Masking

def get_model_timedist(intervals, n_dim=11):
    model = Sequential([
        #Masking(mask_value=0, input_shape=(A, n_dim)), # Masking does not help for some reason (should help with padded data?)
        BatchNormalization(input_shape=(intervals, n_dim), axis=2), # New version of keras doesn't support "mode" attribute, which was used in the original code (mode=0)
        Bidirectional(LSTM(50, return_sequences=False, activation='selu'), input_shape=(intervals, n_dim)),
        Dense(intervals, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy', optimizer='RMSprop',
                  metrics=['binary_accuracy'])
    return model, (-1, intervals, n_dim)


Model from the paper "confused or not confused" with binary output per data point

In [0]:
def get_model(intervals, n_dim=11):
    model = Sequential([
        #Masking(mask_value=0, input_shape=(A, n_dim)), # Masking does not help for some reason (should help with padded data?)
        BatchNormalization(input_shape=(intervals, n_dim), axis=2),
        Bidirectional(LSTM(50, return_sequences=False, activation='selu'), input_shape=(intervals, n_dim)),
        #Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam',
                  metrics=['acc'])
    return model, (-1, intervals, n_dim)

Model from https://github.com/mehmani/DNNs-for-EEG-Signals

In [0]:
def get_mehmani_model(intervals, n_dim=11):
    model = Sequential()
    model.add(TimeDistributed(Conv2D(20, (5,5), activation='relu'),
              input_shape=(1, intervals, n_dim, 1)))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(10, return_sequences=True))
    model.add(Dropout(0.5))
    model.add(Bidirectional(LSTM(20, return_sequences=True)))
    model.add(LSTM(10))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    return model, (-1, 1, intervals, n_dim, 1)

Functions for making the data amount of intervals the same for each data point

In [13]:
def min_max_rows_per_subject_vid(X):
  VideoID = list(set(X[:,1]))
  SubjectID = list(set(X[:,0]))

  max_intervals = 0 # length of signal
  min_intervals = len(X)
  
  for subId in SubjectID:
      for vidId in VideoID:
          X_tmp=X[(X[:, 0] == subId) & (X[:, 1] == vidId)]
          max_intervals = max(len(X_tmp), max_intervals)
          min_intervals = min(len(X_tmp), min_intervals)
  print(max_intervals)
  print(min_intervals)
  assert max_intervals == 144
  return min_intervals, max_intervals

min_intervals, max_intervals = min_max_rows_per_subject_vid(dataset)


def zero_pad_data(X, max_intervals, y_col):
  # Manual Padding to fixed size:
    X_pad = None
    VideoID = list(set(X[:,1]))
    SubjectID = list(set(X[:,0])) 
    for subId in SubjectID:
        for vidId in VideoID:
            X_sv = X[(X[:,0]==subId) & (X[:,1]==vidId)]
            pad_len = max_intervals - X_sv.shape[0]
            
            z = np.zeros((pad_len, X_sv.shape[1]), dtype=X_sv.dtype)
            z[:,0] = X_sv[:,0][pad_len]
            z[:,1] = X_sv[:,1][pad_len]
            z[:,y_col] = X_sv[:,y_col][pad_len]
            
            X_sv_pad = np.concatenate((X_sv, z), axis=0)
            X_sv_pad = X_sv_pad.reshape(1, max_intervals, -1)

            X_pad = X_sv_pad if X_pad is None else np.vstack((X_pad,X_sv_pad))
            
    return X_pad

def truncate_data(X, min_intervals, y_col):
    X_trunc = None
    VideoID = list(set(X[:,1]))
    SubjectID = list(set(X[:,0]))
    for vidId in VideoID:
      for subId in SubjectID:
          X_sv = X[(X[:,0]==subId) & (X[:,1]==vidId)]
          trunc_len = min_intervals
          X_sv_trunc = X_sv[0:trunc_len].reshape(1, min_intervals, -1)
          X_trunc = X_sv_trunc if X_trunc is None else np.vstack((X_trunc, X_sv_trunc))
    return X_trunc




144
112


Define target variable and which variables to use for training

In [0]:

y_col = 13 # The student's confusion column
orig_train_data_cols = list(range(2,14))
vector_cols = list(np.arange(sub_vec_dim) + 15) 

train_cols = orig_train_data_cols
n_dim = len(train_cols)

Cross-validation

In [0]:

from time import time

def train_eval_model(model, X_train, y_train, X_test, y_test, train_cols, intervals,
                     epochs=20, batch_size=20, verbose=1): 
    start = time()
    
    f1s = []
    roc_aucs = []
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                        validation_data=(X_test, y_test), verbose=verbose,
                        shuffle=True,
                        callbacks=[f1_auc_callback(X_test, y_test,
                                                   f1s, roc_aucs)])
    print('model trained in {:.3f} seconds'.format(time() - start))

    loss, acc = model.evaluate(X_test, y_test, verbose=0)

    return (acc, np.nanmean(f1s), np.nanmean(roc_aucs), history)

 
def cross_validate(model, X_shape, data, y_col, train_cols, intervals, even_data, n_test=2,
                   time_distributed=False, verbose=1, epochs=50, batch_size=20):
  """
  even_data: either truncate_data or zero_pad_data to make number of intervals
             even for each data point
  """
  results = []
  initial_weights = model.get_weights()

  for i in range(0, 10, n_test):
    model.set_weights(initial_weights) # Reset weights to forget training done on current iteration's test data  
    
    data_train = even_data(data[np.in1d(data[:,0], (i, i+1), invert=True)], intervals, y_col=y_col)
    data_test = even_data(data[np.in1d(data[:,0], (i, i+1))], intervals, y_col=y_col)
    X_train = data_train[:, :, train_cols]
    y_train = data_train[:, :, y_col]
    X_test = data_test[:, :, train_cols]
    y_test = data_test[:, :, y_col]

    X_train = X_train.reshape(X_shape)
    X_test = X_test.reshape(X_shape)
    if not time_distributed: 
      y_train = y_train.reshape(-1, intervals).mean(axis=1)
      y_test = y_test.reshape(-1, intervals).mean(axis=1)
    
    if verbose > 1: 
      print('Xtrain shape', X_train.shape)
      print('Xtest shape', X_test.shape)
    
      print('ytrain shape', y_train.shape)
      print('ytest shape', y_test.shape)

    start = time()
    print('{}-fold cross validation, iteration {}'
          .format(int(10/n_test), len(results) +1), end=' ')
    acc, f1, roc_auc, history = train_eval_model(model, X_train, y_train,
                                                 X_test, y_test,
                                                 train_cols, intervals,
                                                 epochs=epochs, batch_size=batch_size,
                                                 verbose=verbose)
    
    results.append({'acc': acc, 'F1': f1, 'ROC-AUC': roc_auc})
    if verbose > 0:
      print('current cross-validation mean accuracy: {:.3f}, F1: {:.3f}, and ROC-AUC: {:.3f}'.format(
            *[np.mean([r[key] for r in results]) for key in results[0].keys()]))
      print('iteration time: {:.3f} seconds'.format(time() - start))

  return results



In [0]:
"""
Suppress warnings
"""
import warnings

def warn(*args, **kwargs):
    pass

old_warn = warnings.warn
warnings.warn = warn

In [17]:
"""
Test the model from the paper "Confused or not confused" for pre-defined labels
"""
y_col = 13
train_cols = orig_train_data_cols[:-1]
n_dim = len(train_cols)
model, input_shape = get_model_timedist(min_intervals, n_dim)
tpu_model = tpu_compatibilitate(model)

results = cross_validate(model=tpu_model, X_shape=input_shape, data=dataset,
                         y_col=y_col, train_cols=train_cols, 
                         time_distributed=True, intervals=min_intervals,
                         even_data=truncate_data, n_test=2, verbose=0, epochs=2,
                         batch_size=20)

5-fold cross validation, iteration 1 model trained in 11.694 seconds
5-fold cross validation, iteration 2 model trained in 7.079 seconds
5-fold cross validation, iteration 3 model trained in 7.084 seconds
5-fold cross validation, iteration 4 model trained in 6.992 seconds
5-fold cross validation, iteration 5 model trained in 7.020 seconds


In [18]:
print(*results, sep='\n')
print('cross-validation mean accuracy: {:.3f}, f1: {:.3f}, and roc-auc: {:.3f}'.format(
       *[np.mean([r[key] for r in results]) for key in results[0].keys()]))

{'acc': 0.4906249940395355, 'F1': 0.4733027742346939, 'ROC-AUC': 0.4768454837230628}
{'acc': 0.4950892925262451, 'F1': 0.4925988520408163, 'ROC-AUC': 0.47663118926422954}
{'acc': 0.4928571283817291, 'F1': 0.4996452487244898, 'ROC-AUC': 0.48032936870997256}
{'acc': 0.5133928656578064, 'F1': 0.49435825892857144, 'ROC-AUC': 0.5189761694616064}
{'acc': 0.49866074323654175, 'F1': 0.5015397799744897, 'ROC-AUC': 0.49888442659526994}
cross-validation mean accuracy: 0.498, f1: 0.492, and roc-auc: 0.490


In [0]:
def test_model(name, get_model, y_col, data, time_dist=False, use_sub_vecs=False, truncate=True, epochs=2):
    print('Cross-validation: {}\n'.format(name))
    train_cols = list(range(2,y_col))
    if use_sub_vecs: train_cols += vector_cols
    n_dim = len(train_cols)
    intervals = min_intervals if truncate else max_intervals
    even_data = truncate_data if truncate else zero_pad_data
    
    start = time()
    
    model, input_shape = get_model(intervals, n_dim)
    tpu_model = tpu_compatibilitate(model)
    
    results = cross_validate(model=tpu_model, X_shape=input_shape, data=data,
                         y_col=y_col, train_cols=train_cols, 
                         time_distributed=time_dist,
                         intervals=intervals, even_data=even_data,
                         n_test=2, verbose=0, epochs=epochs, batch_size=20)
    
    print(*results, sep='\n')
    result_summary = '\nCross-validation: {} mean accuracy: {:.3f}, f1: {:.3f}, and roc-auc: {:.3f}'.format(name, 
         *[np.mean([r[key] for r in results]) for key in results[0].keys()])
    print(result_summary)
    print('cross validation total time: {:.4f} min\n'.format((time() - start) / 60))
    return result_summary

In [20]:
result_dict = {}
for modelname, model in {'lstm50': get_model, 'confused': get_model_timedist, 'mehmani': get_mehmani_model}.items():
  for label_col in (13, 14):
    for truncate in (True, False):
      for use_sub_vecs in (False, True):
        label = 'student-defined' if label_col == 14 else 'pre-defined'
        data_evening = 'truncated' if truncate else 'zero padded'
        sub_vec_usage = 'with subtitle vectors' if use_sub_vecs else 'without subtitle vectors'
        
        name = '{} model for {} labels with {} data and {}'.format(
                modelname, label, data_evening, sub_vec_usage)
        time_dist = modelname == 'confused'
        data = dataset if modelname != 'mehmani' else NormDataG
        result_summary = test_model(name, model, label_col, data, time_dist,
                                  use_sub_vecs, truncate, epochs=40)
        result_dict[name] = result_summary

Cross-validation: lstm50 model for pre-defined labels with truncated data and without subtitle vectors

5-fold cross validation, iteration 1 model trained in 140.493 seconds
5-fold cross validation, iteration 2 model trained in 135.637 seconds
5-fold cross validation, iteration 3 model trained in 135.922 seconds
5-fold cross validation, iteration 4 model trained in 136.335 seconds
5-fold cross validation, iteration 5 model trained in 135.889 seconds
{'acc': 0.75, 'F1': 0.9, 'ROC-AUC': 0.7826086956521738}
{'acc': 0.550000011920929, 'F1': 0.5900000000000001, 'ROC-AUC': 0.64}
{'acc': 0.6000000238418579, 'F1': 0.5700000000000001, 'ROC-AUC': 0.6}
{'acc': 0.699999988079071, 'F1': 0.76, 'ROC-AUC': 0.75}
{'acc': 0.30000001192092896, 'F1': 0.32, 'ROC-AUC': 0.3}

Cross-validation: lstm50 model for pre-defined labels with truncated data and without subtitle vectors mean accuracy: 0.580, f1: 0.628, and roc-auc: 0.615
cross validation total time: 11.4516 min

Cross-validation: lstm50 model for pre-

In [21]:
print(*result_dict.values(), sep='\n')


Cross-validation: lstm50 model for pre-defined labels with truncated data and without subtitle vectors mean accuracy: 0.580, f1: 0.628, and roc-auc: 0.615

Cross-validation: lstm50 model for pre-defined labels with truncated data and with subtitle vectors mean accuracy: 0.630, f1: 0.640, and roc-auc: 0.585

Cross-validation: lstm50 model for pre-defined labels with zero padded data and without subtitle vectors mean accuracy: 0.650, f1: 0.694, and roc-auc: 0.626

Cross-validation: lstm50 model for pre-defined labels with zero padded data and with subtitle vectors mean accuracy: 0.700, f1: 0.720, and roc-auc: 0.763

Cross-validation: lstm50 model for student-defined labels with truncated data and without subtitle vectors mean accuracy: 0.510, f1: 0.616, and roc-auc: 0.534

Cross-validation: lstm50 model for student-defined labels with truncated data and with subtitle vectors mean accuracy: 0.510, f1: 0.492, and roc-auc: 0.468

Cross-validation: lstm50 model for student-defined labels wi